In [ ]:
#**********************************************************************************************************
#                OVERVIEW

# Project:       NZ Housing Deprivation Index

# Program Name:  nz_crime_locations

# Summary:       In this program we create a data set which shows the geospatial and temporal descriptors 
#                of all recorded crimes within New Zealand over the 12 month period from 01APR2022 to 
#                31MAR2023.

# Details:       Section 0:
#                User defined variables, required python packages and functions 

#                Section 1:
#                Basic data manipulation 

#                Section 2:
#                Creating a correspondence file that assigns every distinct meshblock that occurrs in the Stats NZ 2023
#                Geographic Areas File to a 2023 meshblock, SA2, SA3 and TA geography. We are required to develop this
#                correspondence file because the 'Meshblock' descriptor in the NZ Police data set is not specific to any 
#                particular year (i.e. some meshblock descriptors may relate to 2023 meshblock boundaries, some to 2022
#                boundaries, some to 2021 boundaries, etc...)

#                Section 3:
#                Merging the Police data with the Meshblock correspondence file

#                Section X:
#                Export CSV Outputs



#**********************************************************************************************************
#                DEPENDENCIES AND LIMITATIONS

# Input(s):      CSV taken from the Victimisations Time and Place section of policedata.nz website:
#                https://www.police.govt.nz/about-us/publications-statistics/data-and-statistics/policedatanz

# Output(s):     AAA_Crime_Locations

# Dependencies:  NA

# Limitations:   NA




In [ ]:
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
# Section 0:
# User defined variables, required python packages and functions 
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------

In [ ]:
# 0.1 - User defined parameters

# Define the folder where the data is stored
crime_folder = r'G:\Academic\My Papers & Thesis\2024 Housing Deprivation Index\Data\Crime Data\\'


# Define the files to read in
crime_data = '20230930 - NZ Police Victimisation Time and Place (36m).csv'


# Define period to consider
start_date = '2022-04-01'
end_date = '2023-03-31'

In [ ]:
# 0.2 - Import required python packages
import csv, re
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import os
from pandas import json_normalize
import datetime



In [ ]:
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
# Section 1:
# Basic data manipulation 
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------

In [ ]:
# Read in the 'NZ Police Victimisation Time and Place' and conducting basic processing

# 1.1a - Read in the latest 'Register' data set
police1a = pd.read_csv(crime_folder+crime_data)


# 1.1b - Refine to required variables and re-order the data set and add a variable called count
police1b = police1a[["ANZSOC Division", "ANZSOC Subdivision", "ANZSOC Group", "Year Month", 
                     "Occurrence Day Of Week", "Occurrence Hour Of Day", "Meshblock", "Victimisations"]]

police1b = police1b.assign(Count=1)


# 1.1c - Rename variables 
police1c = police1b.rename(columns={'ANZSOC Division': 'ANZSOC_Division', 
                                    'ANZSOC Subdivision': 'ANZSOC_Subdivision',
                                    'ANZSOC Group': 'ANZSOC_Group',
                                    'Year Month': 'Date',
                                    'Occurrence Day Of Week': 'DOW',
                                    'Occurrence Hour Of Day': 'Hour'})


# Show output
# police1c



In [ ]:
# Simplifying ANZSOC Subdivisions

# 1.1d - Create a list of all divisions, subdivisions and groups
police1d = police1c[["ANZSOC_Division", "ANZSOC_Subdivision", "ANZSOC_Group"]]
police1d = police1d.drop_duplicates()


# 1.1e - Simplifying ANZSOC subdivision classifications
police1e = police1c

mapping = {'Abduction and Kidnapping': 'Abduction & Kidnapping', 
           'Assault': 'Assault', 
           'Blackmail and Extortion': 'Blackmail & Extortion', 
           'Illegal Use of Property (Except Motor Vehicles)': 'Illegal Use of Property',
           'Motor Vehicle Theft and Related Offences': 'Motor Vehicle Theft',
           'Robbery': 'Robbery',
           'Sexual Assault': 'Sexual Assault',
           'Theft (Except Motor Vehicles)': 'Theft',
           'Unlawful Entry With Intent/Burglary, Break and...': 'Unlawful Entry, B&E'}  # This doesn't map. Taken care of below 

police1e['ANZSOC_Subdivision0'] = police1e['ANZSOC_Subdivision'].map(mapping)

police1e['ANZSOC_Subdivision0'].fillna('Unlawful Entry, B&E', inplace=True)


# 1.1f - Check the correspondence between ANZSOC_Subdivision and ANZSOC_Subdivision0
police1f = police1e[["ANZSOC_Subdivision", "ANZSOC_Subdivision0"]]
police1f = police1f.drop_duplicates()



# 1.1g - Drop old classification, re-order and rename new classification
police1g = police1e.drop(columns=["ANZSOC_Subdivision"])
police1g = police1g.rename(columns={'ANZSOC_Subdivision0': 'ANZSOC_Subdivision'})
police1g = police1g[["ANZSOC_Division", "ANZSOC_Subdivision", "ANZSOC_Group", "Date", "DOW", "Hour",
                    "Meshblock", "Victimisations", "Count"]]




# Show output
# police1f
# police1g



In [ ]:
# Simplifying ANZSOC Groups

# 1.1h - Simplifying ANZSOC Groups classifications
police1h = police1g

mapping = {'Abduction and Kidnapping' : 'Abduction & Kidnapping',
           'Serious Assault Not Resulting in Injury' : 'Serious Assault Not Resulting in Injury',
           'Serious Assault Resulting in Injury' : 'Serious Assault Resulting in Injury',
           'Common Assault' : 'Common Assault',
           'Blackmail and Extortion' : 'Blackmail & Extortion',
           'Illegal Use of Property (Except Motor Vehicles)' : 'Illegal Use of Property',
           'Illegal Use of a Motor Vehicle' : 'Illegal Use of a Motor Vehicle', 
           'Theft of Motor Vehicle Parts or Contents' : 'Theft of Motor Vehicle Parts or Contents',
           'Theft of a Motor Vehicle' : 'Theft of a Motor Vehicle',
           'Aggravated Robbery' : 'Aggravated Robbery',
           'Non-Aggravated Robbery' : 'Non-Aggravated Robbery',
           'Non-Aggravated Sexual Assault' : 'Non-Aggravated Sexual Assault',
           'Aggravated Sexual Assault' : 'Aggravated Sexual Assault',
           'Theft (Except Motor Vehicles), N.E.C.' : 'Theft N.E.C.',
           'Theft From a Person (Excluding By Force)' : 'Theft From a Person',
           'Theft From Retail Premises' : 'Theft From Retail Premises',
           'Unlawful Entry With Intent/Burglary, Break and...' : 'Unlawful Entry, B&E'}  # This doesn't map. Taken care of below 

police1h['ANZSOC_Group0'] = police1h['ANZSOC_Group'].map(mapping)

police1h['ANZSOC_Group0'].fillna('Unlawful Entry, B&E', inplace=True)


# 1.1i - Check the correspondence between ANZSOC_Group and ANZSOC_Group0
police1i = police1h[["ANZSOC_Group", "ANZSOC_Group0"]]
police1i = police1i.drop_duplicates()



# 1.1j - Drop old classification, re-order and rename new classification
police1j = police1h.drop(columns=["ANZSOC_Group"])
police1j = police1j.rename(columns={'ANZSOC_Group0': 'ANZSOC_Group'})
police1j = police1j[["ANZSOC_Division", "ANZSOC_Subdivision", "ANZSOC_Group", "Date", "DOW", "Hour",
                    "Meshblock", "Victimisations", "Count"]]




# Show output
# police1i
# police1j



In [ ]:
# Tidying up Hour classifications and DOW classifications

# 1.1k - Replacing Unknown Hour classifications with 99
police1k = police1j
police1k['Hour'].fillna(99, inplace=True)


# 1.1l - Simplifying day of week classifications
police1l = police1k

mapping = {'Monday' : 'Mon',
           'Tuesday' : 'Tue',
           'Wednesday' : 'Wed',
           'Thursday' : 'Thu',
           'Friday' : 'Fri',
           'Saturday' : 'Sat',
           'Sunday' : 'Sun', 
           'UNKNOWN' : 'Unknown',
           '.' : 'Unknown'}

police1l['DOW0'] = police1l['DOW'].map(mapping)
police1l['DOW0'].fillna('Unknown', inplace=True)



# 1.1m - Drop old classification, re-order and rename new classification
police1m = police1l.drop(columns=["DOW"])
police1m = police1m.rename(columns={'DOW0': 'DOW'})
police1m = police1m[["ANZSOC_Division", "ANZSOC_Subdivision", "ANZSOC_Group", "Date", "DOW", "Hour",
                    "Meshblock", "Victimisations", "Count"]]



# Show output
# police1m

In [ ]:
# Converting the Date variable from string to Datetime format

# 1.1n - Create a function for conversion from string to datetime format
def convert_to_datetime(month_year):
    return pd.to_datetime(month_year, format='%B %Y')

# Applying the function
police1n = police1m
police1n['Date0'] = police1n['Date'].apply(convert_to_datetime)



# 1.1o - Rename the Date variable as Month_Year and Date0 as Date then re-order
police1o = police1n.rename(columns={'Date': 'Month_Year'})
police1o = police1o.rename(columns={'Date0': 'Date'})
police1o = police1o [["ANZSOC_Division", "ANZSOC_Subdivision", "ANZSOC_Group", "Month_Year", "Date",
                     "DOW", "Hour", "Meshblock", "Victimisations", "Count"]]

# Show output
police1o



In [ ]:
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
# Section 2:
# Creating a correspondence file that assigns every distinct meshblock that occurrs in the Stats NZ 2023
# Geographic Areas File to a 2023 meshblock, SA2, SA3 and TA geography. We are required to develop this
# correspondence file because the 'Meshblock' descriptor in the NZ Police data set is not specific to any 
# particular year (i.e. some meshblock descriptors may relate to 2023 meshblock boundaries, some to 2022
# boundaries, some to 2021 boundaries, etc...)
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------

In [ ]:
# Read in the 2023 Geographic Area file

# 2.1a - Read in the 2023 Geographic Area file and refine to required variables
police2a = pd.read_csv(r'G:\Academic\My Papers & Thesis\2024 Housing Deprivation Index\Data\Geographic Area Files\geographic-areas-file-2023.csv')
police2a = police2a[["MB2023_code", "MB2022_code", "MB2018_code", "MB2013_code", 
                     "MB2011_code", "MB2010_code", "MB2006_code", "MB2001_code", 
                     "SA22023_code", "SA22023_name_ascii", 
                     "SA32023_code", "SA32023_name_ascii",
                     "TA2023_code", "TA2023_name_ascii"]]


# 2.1b - Rename variables as per standard conventions
police2b = police2a.rename(columns={'MB2023_code': 'MB2023_Code', 
                                    'MB2022_code': 'MB2022_Code', 
                                    'MB2018_code': 'MB2018_Code', 
                                    'MB2013_code': 'MB2013_Code', 
                                    'MB2011_code': 'MB2011_Code', 
                                    'MB2010_code': 'MB2010_Code', 
                                    'MB2006_code': 'MB2006_Code', 
                                    'MB2001_code': 'MB2001_Code', 
                                    'SA22023_code': 'SA2_2023_Code',
                                    'SA22023_name_ascii': 'SA2_2023_Name',
                                    'SA32023_code': 'SA3_2023_Code',
                                    'SA32023_name_ascii': 'SA3_2023_Name',
                                    'TA2023_code': 'TA2023_Code',
                                    'TA2023_name_ascii': 'TA2023_Name'})


# Show output
# police2b



In [ ]:
# 2.1c - Create 2023 meshblock correspondence

# Refine to required variables
police2c = police2b[["MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]

# Dedup the data set
police2c = police2c.drop_duplicates()

# Create a column called 'Meshblock' and a column called MBYear
police2c['Meshblock'] = police2c['MB2023_Code']
police2c = police2c.assign(MBYear=2023)

# Reorder the data set
police2c = police2c[["MBYear", "Meshblock", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]


# Show Output
# police2c



In [ ]:
# 2.1d - Create 2022 meshblock correspondence

# Refine to required variables
police2d = police2b[["MB2022_Code", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]

# Dedup the data set
police2d = police2d.drop_duplicates()

# Create a column called 'Meshblock' and a column called MBYear
police2d = police2d.rename(columns={'MB2022_Code': 'Meshblock'})
police2d = police2d.assign(MBYear=2022)

# Reorder the data set
police2d = police2d[["MBYear", "Meshblock", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]


# Show Output
# police2d

In [ ]:
# 2.1e - Create 2018 meshblock correspondence

# Refine to required variables
police2e = police2b[["MB2018_Code", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]

# Dedup the data set
police2e = police2e.drop_duplicates()

# Create a column called 'Meshblock' and a column called MBYear
police2e = police2e.rename(columns={'MB2018_Code': 'Meshblock'})
police2e = police2e.assign(MBYear=2018)

# Reorder the data set
police2e = police2e[["MBYear", "Meshblock", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]


# Show Output
# police2e

In [ ]:
# 2.1f - Create 2013 meshblock correspondence

# Refine to required variables
police2f = police2b[["MB2013_Code", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]

# Dedup the data set
police2f = police2f.drop_duplicates()

# Create a column called 'Meshblock' and a column called MBYear
police2f = police2f.rename(columns={'MB2013_Code': 'Meshblock'})
police2f = police2f.assign(MBYear=2013)

# Reorder the data set
police2f = police2f[["MBYear", "Meshblock", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]


# Show Output
# police2f

In [ ]:
# 2.1g - Create 2011 meshblock correspondence

# Refine to required variables
police2g = police2b[["MB2011_Code", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]

# Dedup the data set
police2g = police2g.drop_duplicates()

# Create a column called 'Meshblock' and a column called MBYear
police2g = police2g.rename(columns={'MB2011_Code': 'Meshblock'})
police2g = police2g.assign(MBYear=2011)

# Reorder the data set
police2g = police2g[["MBYear", "Meshblock", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]


# Show Output
# police2g

In [ ]:
# 2.1h - Create 2010 meshblock correspondence

# Refine to required variables
police2h = police2b[["MB2010_Code", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]

# Dedup the data set
police2h = police2h.drop_duplicates()

# Create a column called 'Meshblock' and a column called MBYear
police2h = police2h.rename(columns={'MB2010_Code': 'Meshblock'})
police2h = police2h.assign(MBYear=2010)

# Reorder the data set
police2h = police2h[["MBYear", "Meshblock", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]


# Show Output
# police2h

In [ ]:
# 2.1i - Create 2006 meshblock correspondence

# Refine to required variables
police2i = police2b[["MB2006_Code", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]

# Dedup the data set
police2i = police2i.drop_duplicates()

# Create a column called 'Meshblock' and a column called MBYear
police2i = police2i.rename(columns={'MB2006_Code': 'Meshblock'})
police2i = police2i.assign(MBYear=2006)

# Reorder the data set
police2i = police2i[["MBYear", "Meshblock", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]


# Show Output
# police2i

In [ ]:
# 2.1j - Create 2001 meshblock correspondence

# Refine to required variables
police2j = police2b[["MB2001_Code", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]

# Dedup the data set
police2j = police2j.drop_duplicates()

# Create a column called 'Meshblock' and a column called MBYear
police2j = police2j.rename(columns={'MB2001_Code': 'Meshblock'})
police2j = police2j.assign(MBYear=2001)

# Reorder the data set
police2j = police2j[["MBYear", "Meshblock", "MB2023_Code", "SA2_2023_Code", "SA2_2023_Name", "SA3_2023_Code", "SA3_2023_Name", "TA2023_Code", "TA2023_Name"]]


# Show Output
# police2j

In [ ]:
# 2.1k - Append all the necessary data sets together
police2k = pd.concat([police2c, police2d, police2e, police2f, police2g, police2g, police2i, police2j])

# 2.1l - Sort in ascending order of meshblock, then descending order of MBYear, then ascending order of MB2023_Code
police2l = police2k.sort_values(['Meshblock', 'MBYear', 'MB2023_Code'], ascending = [True, False, True])

# 2.1m - Enumerate the data set by Meshblock
police2m = police2l
police2m['Count'] = police2m.groupby(['Meshblock']).cumcount()

# 2.1n - Refine to the first instance of each meshblock
police2n = police2m[police2m['Count'] == 0]

# If done correctly should have mostly meshblocks from 2023
police2n1 = police2n['MBYear'].value_counts()

# 2.1o - Drop non-required variables
police2o = police2n.drop(columns=["MBYear", "Count"])




# show output
police2o


In [ ]:
# The police2o data set effectively assigns every meshblock that occurs in the 2023 Stats NZ geographic area file to a 2023 
# meshblock and associated higher geographies. However, we note that meshblocks get split over time. For instance, between 
# 2001 and 2023 the 2001 meshblock
#        MB2001 = 1203
# gets split into two 2023 meshblocks
#       MB2023 = {4011710, 4011711} 
# both of which are contained in the 2023 SA2 labelled SA2_2023 = 100100.

# When there are multiple 2023 meshblocks that a meshblock (as recorded by NZ Police) can be mapped to, we will simply take
# the meshblock that occurrs first alpha-numerically along with its associated higher geographies.

# From the perspective of geo-locating crimes (i.e. plotting their location on a map) this is perfectly fine 
# as the centroids of the 2023 meshblocks provide a decent approximation to the centroid of the original meshblock, the
# 2023 meshblocks being a geographic subset of the original meshblock.

# However, this can cause reporting issues in the rare case whereby meshblocks are split across different SA2s. For instance,
# between 2001 and 2023, the 2001 meshblock
#        MB2001 = 2300
# gets split into five 2023 meshblocks,
#        MB2023 = {2301, 2302, 2303, 2305, 2306}

# Whilst the first four meshblocks belong to 2023 SA2 100200, the last meshblock belongs to the 2023 SA2 100500. Thus, there
# are multiple 2023 SA2s to which a crime said occur in MB2001 = 2300 could be attributed. 

# We will try and understand the error that this incurrs within our analysis.



In [ ]:
 # Identify instances of meshblocks being split across multiple 2023 SA2s

# Create a copy of the police2l data set and refine to required variables, then de-dup
sa2_split1a = police2l[["Meshblock", "SA2_2023_Code"]]
sa2_split1a = sa2_split1a.drop_duplicates()

# Count the number of times each Meshblock occurs
sa2_split1b = sa2_split1a['Meshblock'].value_counts()
sa2_split1b = sa2_split1b.reset_index()
sa2_split1b = sa2_split1b.rename(columns={'Meshblock': 'MB_Split_Count'})
sa2_split1b = sa2_split1b.rename(columns={'index': 'Meshblock'})


# Merge back together
sa2_split1c = pd.merge(sa2_split1a, sa2_split1b, on="Meshblock", how='left')

# Identify instances of meshblock splitting
sa2_split1d = sa2_split1c[sa2_split1c['MB_Split_Count'] > 1]

# Refine to required variables, then de-duping
sa2_split1e = sa2_split1d[["Meshblock", "MB_Split_Count"]]
sa2_split1e = sa2_split1e.drop_duplicates()
sa2_split1e = sa2_split1e.assign(MB_Split_Flag=1)
sa2_split1e = sa2_split1e[["Meshblock", "MB_Split_Flag", "MB_Split_Count"]]


# show output
# sa2_split1e

num_rows = len(sa2_split1e)
print("Number of rows:", num_rows)



In [ ]:
# The above analysis indicates that of the 67,677 distinct Meshblocks that occurr in the Stats NZ 2023 Geographic area file,
# approximately 2.4% get mapped to multiple SA2s. Of course, we need to assess what prportion of crimes actually occurr in
# these rare meshblocks.



In [ ]:
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
# Section 3:
# Merging the Police data with the Meshblock correspondence file
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------

In [ ]:
# Merging the Polic data with the Meshblock correspondence file

# 3.1a - Conducting a left merge with the relevant data sets
police3a = pd.merge(police1o, police2o, on="Meshblock", how='left')
police3a = police3a[["TA2023_Code", "TA2023_Name", 
                     "SA3_2023_Code", "SA3_2023_Name", 
                     "SA2_2023_Code", "SA2_2023_Name", 
                     "MB2023_Code", "Meshblock", 
                     "ANZSOC_Division", "ANZSOC_Subdivision", "ANZSOC_Group", 
                     "Month_Year", "Date", "DOW", "Hour", 
                     "Victimisations", "Count"]]


# 3.1a1 - Refine to all crimes that occurred between start_date and end_date
police3a1 = police3a[(police3a['Date'] >= start_date) & (police3a['Date'] <= end_date)]


# 3.1b - Removing all records where the 'Meshblock' has not bee mapped to a 2023 Meshblock
police3b = police3a1
police3b = police3b.dropna(subset=['MB2023_Code'])
police3b = police3b.drop(columns=["Meshblock"])


# 3.1c - Convert all numeric geographic indicators to integer type
police3c = police3b
numeric_columns = ['TA2023_Code', 'SA3_2023_Code', 'SA2_2023_Code', 'MB2023_Code']
police3c[numeric_columns] = police3c[numeric_columns].astype(int)


# 3.1d - Create an alternative Date format
police3d = police3c
police3d['Date1'] = police3d['Date'].dt.strftime('%d-%b-%Y')
police3d = police3d[["TA2023_Code", "TA2023_Name", 
                     "SA3_2023_Code", "SA3_2023_Name", 
                     "SA2_2023_Code", "SA2_2023_Name", 
                     "MB2023_Code",  
                     "ANZSOC_Division", "ANZSOC_Subdivision", "ANZSOC_Group", 
                     "Month_Year", "Date", "Date1", "DOW", "Hour", 
                     "Victimisations", "Count"]]



# Show Output
# police3d



In [ ]:
# There are 285,170 rows in the police3a1 data set.
# There are 282,931 rows in the police3b data set.
# This means that there are 2,239 (0.79%) crimes in the police3a1 data set that cannot be mapped to a meshblock.
# Below we refine these estimates for crimes whose ANZSOC Group is defined as 'Unlawful Entry, B&E'

# Calculate how many instances of B&E in the police3a1 data set
count_unlawful_entry = police3a1['ANZSOC_Group'].value_counts().get('Unlawful Entry, B&E', 0)
print("Number of rows with 'Unlawful Entry, B&E in police3a1':", count_unlawful_entry)

# Calculate how many instances of B&E in the police3b data set
count_unlawful_entry = police3b['ANZSOC_Group'].value_counts().get('Unlawful Entry, B&E', 0)
print("Number of rows with 'Unlawful Entry, B&E in police3b':", count_unlawful_entry)

# Thus, there are 218 (0.32%) instances of B&E that could not be mapped to a 2023 Meshblock of occurrence

In [ ]:
# Merge in meshblock 2023 centroid information

# 3.1e - Read in the 2023 meshblock centroid information
police3e = pd.read_csv(r'G:\Academic\My Papers & Thesis\2024 Housing Deprivation Index\Data\Geographic Area Files\meshblock-2023-centroid-inside.csv')
police3e = police3e[["MB2023_Code", "LANDWATER_NAME", "Latitude", "Longitude"]]
police3e = police3e.rename(columns={'LANDWATER_NAME': 'Meshblock_Class'})


# 3.1f - Merge with the previous data set and re-order
police3f = pd.merge(police3d, police3e, on="MB2023_Code", how='left')
police3f = police3f[["TA2023_Code", "TA2023_Name", 
                     "SA3_2023_Code", "SA3_2023_Name", 
                     "SA2_2023_Code", "SA2_2023_Name", 
                     "MB2023_Code",  "Meshblock_Class", "Longitude", "Latitude", 
                     "ANZSOC_Division", "ANZSOC_Subdivision", "ANZSOC_Group", 
                     "Month_Year", "Date", "Date1", "DOW", "Hour", 
                     "Victimisations", "Count"]]


# 3.1g - Count the number of instances of each meshblock class (Mainland, Inlet, Island, Oceanic, Inland Water)
police3g = police3f.groupby(['Meshblock_Class', 'ANZSOC_Group'])['Count'].sum().reset_index()



# 3.1h - Add DOW_Order variable then re-order
police3h = police3f

dow_mapping = {'Mon': 1, 
           'Tue': 2, 
           'Wed': 3, 
           'Thu': 4,
           'Fri': 5,
           'Sat': 6,
           'Sun': 7, 
           'Unknown': 8}

police3h['DOW_Order'] = police3h['DOW'].map(dow_mapping)

police3h = police3h[["TA2023_Code", "TA2023_Name", 
                     "SA3_2023_Code", "SA3_2023_Name", 
                     "SA2_2023_Code", "SA2_2023_Name", 
                     "MB2023_Code",  "Meshblock_Class", "Longitude", "Latitude", 
                     "ANZSOC_Division", "ANZSOC_Subdivision", "ANZSOC_Group", 
                     "Month_Year", "Date", "Date1", "DOW", "DOW_Order", "Hour", 
                     "Victimisations", "Count"]]


# Show output
police3h

In [ ]:
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
# Section X:
# Export CSV Outputs
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------------------------------------

In [ ]:
# X.1a - Create final outputs
A1a_Crime_Locations = police3h

# X.1b - Export the 'Proxy Vehicle Mapping' data
A1a_Crime_Locations.to_csv(r'G:\Academic\My Papers & Thesis\Power BI\Power BI Data\A1a_Crime_Locations.csv')

